In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from sklearn.metrics import mean_squared_error
%matplotlib inline
np.set_printoptions(suppress=True)   
pd.set_option('display.float_format', lambda x: '%.5f' % x)  
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)
import pandas as pd
import re
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns
%matplotlib inline
np.set_printoptions(suppress=True)   
import sklearn
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestRegressor
from matplotlib.ticker import FuncFormatter
from matplotlib.pyplot import MultipleLocator
import pickle
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
import joblib
import scipy.stats
from time import *
from tqdm import tnrange, tqdm_notebook
from time import sleep
from quantile_forest import RandomForestQuantileRegressor

In [ ]:
var = locals()

In [ ]:
RF = joblib.load('./RF_V6_alldatatraining_100bootstrap_1000Nmax.pkl')
pt = joblib.load('./boxcox_V6_alldatatraining_100bootstrap_1000Nmax.pkl')
for i in tqdm_notebook(range(5,105)):
    var['RF_'+str(i)] = joblib.load('./RF_V6_alldatatraining_100bootstrap_1000Nmax_'+ str(i) +'.pkl')

In [ ]:
nh3 = pd.read_excel('./df_training.xlsx')

In [ ]:
nh3_orig = nh3.copy()

In [ ]:
###

In [ ]:
crops = nh3['Crop type'].unique()
fers = nh3['Fertilizer type'].unique()
nplaces = nh3['Nitrogen placement'].unique()
tills = [0,1]
ntimes = [0,1]
t_name = ['CT','NT']
n_name = ['one','more']

In [ ]:
X_col = ['FAT',
 'STP',
 'Water',
 'Tem',
 'SOC',
 'TN',
 'pH',
 'BD',
 'Clay',
 'CEC',
 'Nrate',
 'UOA',
 'others',
 'CRF',
 'manure',
 'AN',
 'SBC',
 'Mix',
 'DPM',
 'Rice',
 'Wheat',
 'Maize']

In [ ]:
dfs = ['rice1', 'rice2','wheat1','wheat2','maize1','maize2']

In [ ]:
begin_time = time()

In [ ]:
df_mean = pd.DataFrame(columns=['Ctype','Ftype','NP','FAT','STP','EF2018_mean','EF2018_5','EF2018_95','EF2018_std','EF2018_min','EF2018_max'])
Ctype = []
Ftype = []
NP = []
FAT = []
STP = []
EF2018_mean = []
EF2018_5 = []
EF2018_95 = []
EF2018_std = []
EF2018_min = []
EF2018_max = []
for df in dfs:
    var[df] = pd.read_csv('./' + df + '.csv')
    var[df]['water'] = var[df]['pre'] + var[df]['iri']
    for fer in tqdm_notebook(fers):
        for nplace in nplaces:
            for till in tills:
                for ntime in ntimes:
                    ds = pd.DataFrame(np.zeros((len(var[df]),len(X_col))), columns=X_col)
                    ds[var[df].columns] = var[df][var[df].columns]
                    ds[df[0:-1].capitalize()] = 1
                    ds[fer] = 1
                    ds[nplace] = 1
                    ds['STP'] = till
                    ds['FAT'] = ntime
                    ds[['Water','Tem','SOC','TN','pH','BD','Clay','CEC','Nrate']] = ds[['water','tem','soc','tn','ph','bd','clay','cec','Nrate_2018']]
                    ds.loc[ds['Nrate'] > 1000, 'Nrate'] = 1000
                    sdf = pd.DataFrame()
                    for i in tqdm_notebook(range(5,105)):
                        sdf[str(i)] = var['RF_'+str(i)].predict(ds[X_col])
                    ds['BoxEF_2018'] = np.mean(sdf.iloc[:,:100], axis = 1)
                    ds['BoxEF_2018_025'] = np.percentile(sdf.iloc[:,:100], 2.5 ,axis =1)
                    ds['BoxEF_2018_975'] = np.percentile(sdf.iloc[:,:100], 97.5 ,axis =1)
                    #sdf['std'] = np.std(sdf.iloc[:,:100], axis = 1)
                    ds['EF_2018'] = pt.inverse_transform(np.array(ds['BoxEF_2018']).reshape(-1, 1))
                    ds['EF_2018_025'] = pt.inverse_transform(np.array(ds['BoxEF_2018_025']).reshape(-1, 1))
                    ds['EF_2018_975'] = pt.inverse_transform(np.array(ds['BoxEF_2018_975']).reshape(-1, 1))
                    ds = ds[ds.columns.drop(['water','tem','soc','tn','ph','bd','clay','cec'])]

                    ds.to_csv('./' + df+'_'+fer+'_'+nplace+'_'+t_name[till]+'_'+n_name[ntime] + '_alldatatraining_100bootstrap_1000Nmax.csv', index = False)

In [ ]:
end_time = time()
run_time = end_time-begin_time
print ('time cost：',run_time/60, 'min') 